# Pokemon Attack Scraping Script

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup




physical_moves = "https://www.serebii.net/attackdex-swsh/physical.shtml" 
special_moves = "https://www.serebii.net/attackdex-swsh/special.shtml"
status_moves = "https://www.serebii.net/attackdex-swsh/other.shtml"

In [4]:


data = requests.get(physical_moves)

In [29]:
my_data = []
 #https://www.kite.com/python/examples/4420/beautifulsoup-parse-an-html-table-and-write-to-a-csv
html = BeautifulSoup(data.text, 'html.parser')


In [83]:

soup = html
table = soup.find_all("table")[1]

output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text.strip())
    output_rows.append(output_row)



In [84]:
moves = pd.DataFrame(output_rows)

In [85]:
moves.columns = moves.iloc[0]

In [86]:
moves = moves[1:]

In [87]:
moves = moves[moves.Effect.apply(lambda x: "This move can't be used" not in  x)]

In [88]:
moves

,Name,Type,Cat.,PP,Att.,Acc.,Effect
1,Accelerock,,,20,40,100,The user smashes into the target at high speed...
3,Acrobatics,,,15,55,100,The user nimbly strikes the target. If the use...
4,Aerial Ace,,,20,60,101,"The user confounds the target with speed, then..."
6,Anchor Shot,,,20,80,100,The user entangles the target with its anchor ...
7,Aqua Jet,,,20,40,100,The user lunges at the target at a speed that ...
...,...,...,...,...,...,...,...
381,Wood Hammer,,,15,120,100,The user slams its rugged body into the target...
382,Wrap,,,20,15,90,"A long body, vines, or the like are used to wr..."
383,X-Scissor,,,15,80,100,The user slashes at the target by crossing its...
384,Zen Headbutt,,,15,80,90,The user focuses its willpower to its head and...


In [95]:
def create_moves_df(url):
    # given a url, scrapes the moves table and turns it into a pandas df
    # works on Serebii's moves attackdex
    # removes moves that no longer are usable in that generation
    data = requests.get(url)
    soup = BeautifulSoup(data.text, 'html.parser')
    
    # first table is the table of contents for the moves, we don't want that
    
    # following code grabbed from   #https://www.kite.com/python/examples/4420/beautifulsoup-parse-an-html-table-and-write-to-a-csv
    table = soup.find_all("table")[1]

    output_rows = []
    for table_row in table.findAll('tr'):
        columns = table_row.findAll('td')
        output_row = []
        for column in columns:
            output_row.append(column.text.strip())
        output_rows.append(output_row)
    
    moves = pd.DataFrame(output_rows)
    
    # move first row to column, and drop row
    moves.columns = moves.iloc[0]
    moves = moves[1:].reset_index(drop = True)
    
    # drop moves that don't exist in this gen, but also count them
    unusable_moves = len(moves.Effect.apply(lambda x: "This move can't be used" in  x))
    print("Removing some old moves... Found ", unusable_moves)
    moves = moves[moves.Effect.apply(lambda x: "This move can't be used" not in  x)]
    moves = moves.replace("--", 0)
    return moves




physical_moves = "https://www.serebii.net/attackdex-swsh/physical.shtml" 
special_moves = "https://www.serebii.net/attackdex-swsh/special.shtml"
status_moves = "https://www.serebii.net/attackdex-swsh/other.shtml"


physical_df = create_moves_df(physical_moves)
special_df = create_moves_df(special_moves)
status_df = create_moves_df(status_moves)

moves = pd.concat([physical_df, special_df, status_df])
moves.to_csv("data/moves.csv")

Removing some old moves... Found  386
Removing some old moves... Found  237
Removing some old moves... Found  260


,Name,Type,Cat.,PP,Att.,Acc.,Effect
0,Accelerock,,,20,40,100,The user smashes into the target at high speed...
2,Acrobatics,,,15,55,100,The user nimbly strikes the target. If the use...
3,Aerial Ace,,,20,60,101,"The user confounds the target with speed, then..."
5,Anchor Shot,,,20,80,100,The user entangles the target with its anchor ...
6,Aqua Jet,,,20,40,100,The user lunges at the target at a speed that ...
...,...,...,...,...,...,...,...
255,Withdraw,,,40,0,101,The user withdraws its body into its hard shel...
256,Wonder Room,,,10,0,101,The user creates a bizarre area in which Pokém...
257,Work Up,,,30,0,101,"The user is roused, and its Attack and Sp. Atk..."
258,Worry Seed,,,10,0,100,A seed that causes worry is planted on the tar...
